<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_Agnostic_Run.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Run Agnostic App  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This path will run the executable you want, with some bells and whitles.

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. 

This is a practical walkthrough for defining and deploying an **HPC** app using the Tapis v3 API.


## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
import json

In [2]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


---
## Connect to Tapis

In [3]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-21T21:43:46+00:00
 Token expires in: 3:57:34.136821
-- LOG IN SUCCESSFUL! --


---
## Configure App

In [4]:
app_id = 'agnostic-app-test'

#### List the app schema

In [5]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
OpsUtils.display_tapis_app_schema(appMetaData)

########################################
########### TAPIS-APP SCHEMA ###########
########################################
######## appID: agnostic-app-test
######## version: 0.0.29
########################################
{
  sharedAppCtx: "silvia"
  isPublic: False
  tenant: "designsafe"
  id: "agnostic-app-test"
  version: "0.0.29"
  description: "Run any Executable"
  owner: "silvia"
  enabled: True
  versionEnabled: True
  locked: False
  runtime: "ZIP"
  runtimeVersion: None
  runtimeOptions: None
  containerImage: "tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.29/agnostic-app-test.zip"
  jobType: "BATCH"
  maxJobs: 2147483647
  maxJobsPerUser: 2147483647
  strictFileInputs: False
  uuid: "e78ead3a-90f4-44ac-b41f-30f59d2486af"
  deleted: False
  created: "2025-08-21T05:42:14.691906Z"
  updated: "2025-08-21T05:42:14.691906Z"
  sharedWithUsers: []
  tags: ["portalName: DesignSafe", "portalName: CEP"]
  jobAttributes: {
    description: None
    dynamicExecSys

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

---
### Initialize

In [6]:
tapisInputAll = {}

---
### SLURM-Specific Input

In [7]:
tapisInputAll["maxMinutes"] = 6

tapisInputAll["execSystemId"] = "stampede3"
tapisInputAll["execSystemLogicalQueue"] = "skx-dev"
tapisInputAll["nodeCount"] = 1
tapisInputAll["coresPerNode"] = 48
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='Work' # Options: MyData or Work

---
### App-Specific Input

In [8]:
print('app_id:',app_id)

app_id: agnostic-app-test


### OpenSees

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'OpenSees'
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-express',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- OpenSees_opensees-express --

Creating job_description
job_description {'name': 'OpenSees_opensees-express', 'maxMinutes': 6, 'appId': 'opensees-express', 'appVersion': 'latest', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'envVariables': [{'key': 'mainProgram', 'value': 'OpenSees'}, {'key': 'tclScript', 'value': 'Ex1a.Canti2D.Push.tcl'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}'}
Submitting Job
Job submitted! ID: 1c089ade-a880-49c8-b152-f4625b503a82-007
job_start_time: 1755798380.8641062

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.1 sec	 Current Status: PENDING
	 Elapsed job time: 2.13 sec	 Current Status: PROCESSING_INPUTS		(PENDING took 1.03 sec)
	 Elapsed job time: 7.4 sec	 Current Status: STAGING_IN

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (1c089ade-a880-49c8-b152-f4625b503a82-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (1c089ade-a880-49c8-b152-f4625b503a8…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (1c089ade-a880-49c8-b152-f4625b5…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (1c089ade-a880-49c8-b152-f4625b503a8…


 -- OpenSees_agnostic-app-test --

Creating job_description
job_description {'name': 'OpenSees_agnostic-app-test', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'OpenSees'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.tcl'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'False'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}'

### OpenSeesMP

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'OpenSeesMP'
tapisInput['Main Script'] = 'Ex1a_many.Canti2D.Push.mp.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-mp-s3',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesSP

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'OpenSeesSP'
tapisInput['Main Script'] = 'simpleSP.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-sp-s3',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesPy

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'python'
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesPy - mpi

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'python'
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.mpi.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------

### OpenSeesPy - mpi4py

In [ ]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'python'
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.mpi4py.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------